In [3]:
#importing the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sqlite3
import pickle  
from sklearn.feature_extraction.text import CountVectorizer  ## BOW Model
from sklearn.feature_extraction.text import TfidfVectorizer  ## TFIDF Model
from sklearn.manifold import TSNE  
import re
import nltk
from nltk.corpus import stopwords
import gensim

ModuleNotFoundError: No module named 'gensim'

### PREPROCESSING START

#### IMPORTING THE FINAL SQLITE FILE

In [ ]:
con=sqlite3.connect("final2.sqlite")
final=pd.read_sql_query("select * from Reviews",con)
con.close()

#### CHEAKING THE FIRST TWO VALUES

In [1]:
final2.head(2) #printing first two rows

NameError: name 'final2' is not defined

In [19]:
del final["level_0"] #deleting the unrequired column

In [20]:
final.shape # 12 columns,364k rows

(364171, 12)

### CHEAKING THE FIRST TWO SUMMARY BEFORE PREPROCESSING

In [21]:
i = 0
for sen in final['Summary'].values:
    if(len(re.findall('<.*?>', sen))):
        print(sen,"\n\n")
        i += 1
    if i == 5:
        break
        
# in first five rows there is no html tag

#### REMOVAL OF STOP WORDS

In [22]:
stop = set(stopwords.words('english')) #set of stopwords which are of no use.
# they will only increse the dimentionality

In [23]:
lst = ['won', 'nor', 'not', 'against']
for word in lst:
    stop.remove(word)
print(stop)

{'their', 'while', 'any', 'been', 'our', "mustn't", "don't", 'ourselves', 'here', 'few', "hadn't", "wouldn't", 'wouldn', 've', "couldn't", 'does', 'shouldn', 'too', 'didn', 'yourselves', 'now', 'for', 'themselves', 'yours', 'am', 'so', 'out', 'other', 'an', 'her', 'shan', 'no', 'above', 'doesn', "it's", 'his', 'isn', 'through', 'yourself', "you've", "shan't", 'him', 'y', 'after', 'i', 'is', 'needn', 'them', 'why', 'hers', 'only', 'below', 'this', 'm', 'by', 'will', "should've", 'haven', 'these', 'be', 'weren', 'on', 'which', 're', 'that', 'was', 'do', 'in', 'most', "shouldn't", 'or', 'being', 'have', 'such', 'are', 'o', 'wasn', "doesn't", 'can', "that'll", 'same', 'having', 'because', 'theirs', 'just', 'we', 'he', 'my', "isn't", 'who', 'off', 't', 'it', 'had', 'very', 'down', 'a', 'and', 's', 'did', "aren't", 'couldn', 'hasn', "she's", 'under', 'whom', 'where', "needn't", "you're", 'should', 'own', 'to', 'hadn', 'how', 'doing', 'further', 'those', 'don', 'until', 'if', 'the', 'ain', 'b

### STEMMING 
#### which are use to give a same title for different words having same meaning.

In [24]:
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer
#the above function will used to remove the stems in english word.

### CLEANING OF HTML TAGS
#### this function will remove the different tags which are accosiated with the SUMMARY.

In [25]:
def cleanhtml(sentence):
    '''This function removes all the html tags in the given sentence'''
    cleanr = re.compile('<.*?>')    ## find the index of the html tags
    cleantext = re.sub(cleanr, ' ', sentence)  ## Substitute <space> in place of any html tag
    return cleantext

### REMOVING THE PUNCTUATION 
#### the punctuation are of not use in our analysis. it just increase the dimentionality.

In [26]:
def cleanpunc(sentence):
    '''This function cleans all the punctuation or special characters from a given sentence'''
    cleaned = re.sub(r'[?|@|!|^|%|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

## PREPROCESSING

In [27]:
def preprocessing(series):
   
    i = 0
    str1=" "
    final_string = []    ## This list will contain cleaned sentences
    list_of_sent = []    ## This is a list of lists used as input to the W2V model at a later stage
    
    ## Creating below lists for future use
    all_positive_words=[] # store words from +ve reviews here
    all_negative_words=[] # store words from -ve reviews here
    
    
    for sent in series.values:
        ## 
        filtered_sent = []
        sent = cleanhtml(sent)    ## Clean the HTML tags
        sent = cleanpunc(sent)    ## Clean the punctuations and special characters
        ## Sentences are cleaned and words are handled individually
        for cleaned_words in sent.split():
            ## Only consider non-numeric words with length at least 3
            if((cleaned_words.isalpha()) and (len(cleaned_words) > 2)):
                ## Only consider words which are not stopwords and convert them to lowet case
                if(cleaned_words.lower() not in stop):
                    ## Apply snowball stemmer and add them to the filtered_sent list
                    s = (sno.stem(cleaned_words.lower()))#.encode('utf-8')
                    filtered_sent.append(s)    ## This contains all the cleaned words for a sentence
                    if (final['Score'].values)[i] == 'positive':
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews
        ## Below list is a list of lists used as input to W2V model later
        list_of_sent.append(filtered_sent)
        ## Join back all the words belonging to the same sentence
        str1 = " ".join(filtered_sent)
        ## Finally add the cleaned sentence in the below list
        final_string.append(str1)
        #print(i)
        i += 1
    return final_string, list_of_sent

#### the above fuction will return the preprocessed or cleaned text.
#### it is also return the nested list which containing the element of the sentence in sub-list.

In [28]:
## This takes around 2-3 minutes
final_string, list_of_sent = preprocessing(final['Summary'])# we have pass the Summary column to preproccesing.

In [29]:
final.shape

(364171, 12)

### ACCESSING THE ELEMNT OF FINAL STRING

In [30]:
final_string[75000]

'coconut oil'

### ACCESSING THE ELEMNT OF NESTED LIST

In [31]:
list_of_sent[75000]

['coconut', 'oil']

### adding the preprocceing into dataset

In [32]:
final['CleanedSummary']=final_string

In [33]:
final.shape #cheaking the final shape

(364171, 13)

### Saving List Of Sentence for further Use

In [67]:
with open('Summary_list_of_sent.pkl', 'wb') as pickle_file:
    pickle.dump(list_of_sent, pickle_file)

In [2]:
with open('Summary_list_of_sent.pkl', 'rb') as pickle_file:
    list_of_sent=pickle.load( pickle_file)

### SAVING THE FINAL SQLITE FILE

In [85]:
conn = sqlite3.connect('Summary_final1.sqlite')
c=conn.cursor()
final.to_sql('Reviews', conn, if_exists='replace', index = False)
conn.close()

### LOADING THE FILE TO FOR FURTHER USE

In [4]:
con=sqlite3.connect("Summary_final.sqlite")
final=pd.read_sql_query("select * from Reviews",con)
con.close()

#### SHUFFLING OF DATASET WITH RANDOM STATE 123

In [31]:
from sklearn.utils import shuffle
fina=shuffle(final ,n_samples=364171, random_state=123)

In [92]:
fina.head(5) #cheaking first two rows


,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Cleaned_Text,CleanedSummary
253564,499556,540125,B002W08W56,A2STPOZAT37RNE,Betty Baker,1,1,1,1300320000,"Wonderful, but surprised to find this size can...",Great beans....the best texture hands down. T...,great bean best textur hand seem great tast do...,wonder surpris find size
123620,29129,31763,B000UXWQMC,A115Y8R40DCT3I,Guy,0,0,1,1345161600,better than expected,I usually buy whole pitted olives and when I b...,usual buy whole pit oliv bought good wasnt who...,better expect
111670,471146,509504,B000P09RJA,A2BR8LGC8N6XQD,dj2u,0,4,0,1278374400,Wrong size bars sent,The first shipment was just as ordered. This ...,first shipment order last shipment not bar wen...,wrong size bar sent
91849,25391,27728,B000K8ESBY,A1RYTIMIWPDJ4O,"Bill Ison ""Bill""",0,0,1,1256169600,Coffee flavoring,"<a href=""http://www.amazon.com/gp/product/B000...",flavour creation coffe flavor tablet french va...,coffe flavor
155724,118631,128657,B001AVJT0K,AJ6SZ4YAPOOO7,Mwebi,0,0,1,1348099200,They all come running,"I have the pickiest cats, they hate 99/100 kin...",pickiest cat hate kind food better vari otherw...,come run


#### RESET THE INDEX BECAUSE IT WILL CREATE PROBLEM DURING CONCATINATION OF TWO DATAFRAMES

In [83]:
final=fina.reset_index(drop=True)

In [84]:
final.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Cleaned_Text,CleanedSummary
0,499556,540125,B002W08W56,A2STPOZAT37RNE,Betty Baker,1,1,1,1300320000,"Wonderful, but surprised to find this size can...",Great beans....the best texture hands down. T...,great bean best textur hand seem great tast do...,wonder surpris find size
1,29129,31763,B000UXWQMC,A115Y8R40DCT3I,Guy,0,0,1,1345161600,better than expected,I usually buy whole pitted olives and when I b...,usual buy whole pit oliv bought good wasnt who...,better expect
2,471146,509504,B000P09RJA,A2BR8LGC8N6XQD,dj2u,0,4,0,1278374400,Wrong size bars sent,The first shipment was just as ordered. This ...,first shipment order last shipment not bar wen...,wrong size bar sent
3,25391,27728,B000K8ESBY,A1RYTIMIWPDJ4O,"Bill Ison ""Bill""",0,0,1,1256169600,Coffee flavoring,"<a href=""http://www.amazon.com/gp/product/B000...",flavour creation coffe flavor tablet french va...,coffe flavor
4,118631,128657,B001AVJT0K,AJ6SZ4YAPOOO7,Mwebi,0,0,1,1348099200,They all come running,"I have the pickiest cats, they hate 99/100 kin...",pickiest cat hate kind food better vari otherw...,come run


### SHUFFLING THE LIST FOR GETTING BETTER RESULTS

In [13]:
from sklearn.utils import shuffle
finall=shuffle(list_of_sent ,n_samples=364171, random_state=123)

In [15]:
finall[0:5]

[['wonder', 'surpris', 'find', 'size'],
 ['better', 'expect'],
 ['wrong', 'size', 'bar', 'sent'],
 ['coffe', 'flavor'],
 ['come', 'run']]

### TFIDF weighted average Word2Vec for summary
#### HERE WE ARE self CREATED W2V MODEL

In [20]:
def calc_tfidf_avg_w2v(list_of_sent, w2v_model, tf_idf, tfidf_feat):
    '''This function takes in 4 parameters as follows:
       1. list_of_sent - This is the list of sentences/reviews for which sentence vetors are to be constructed
       2. w2v_model - This is the Word2Vec model which is trained on the working corpus - contains the word vectors
       3. tf_idf - This is the TF-IDF model built using the same reviews/sentences - it is the TF-IDF sparse matrix
       4. tfidf_feat - This is the feature vector constructed from the TF-IDF model
       
       Return Value:
       tfidf_sent_vectors - This is a list of sentence/review vectors constructed by using tfidf weighted average on the word vectors
    '''
    
    ## Initialize an empty list
    tfidf_sent_vectors = []
    row = 0
    ## Consider one sentence/review at a time
    for sent in list_of_sent:
        ## Initialize sentence vector to 0
        sent_vec = np.zeros(50)
        ## Initialize weighted sum to 0
        weighted_sum = 0
        ## Consider the words one by one
        for word in sent:
            try:
                ## Calculate the word vector using the W2V model
                vec = w2v_model.wv[word]
                ## Calculate tfidf value of the word in that review using tfidf model
                tfidf = tf_idf[row, tfidf_feat.index(word)]
                ## Add the product of tfidf*word_vec to the sentence vector (This is the numerator)
                sent_vec += vec*tfidf
                ## Sum all the tfidf values (This is the denominator)
                weighted_sum += tfidf
            except:
                pass
        #print(row, weighted_sum)
        
        ## Divide the numerator by the denominator to get the sentence vector
        sent_vec /= weighted_sum
        ## Add the sentence vector in the final list
        tfidf_sent_vectors.append(sent_vec)
        row += 1
    ## return the list of all the sentence vectors
    return tfidf_sent_vectors

#### creating tfidf for above function

In [21]:
tf_idf_vect = TfidfVectorizer()
tf_idf = tf_idf_vect.fit_transform(final['CleanedSummary'].values)
tfidf_feat = tf_idf_vect.get_feature_names()
#w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)

### CREATING A 50 DIMENSIONAL W2V MODEL

In [25]:
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)

In [91]:
type(w2v_model)

gensim.models.word2vec.Word2Vec

In [52]:
len(tfidf_feat)

23177

In [56]:
type(tf_idf)

scipy.sparse.csr.csr_matrix

In [26]:
tfidf_sent_vectors = calc_tfidf_avg_w2v(list_of_sent[0:60000], w2v_model, tf_idf, tfidf_feat)

C:\Users\harinarayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide


In [27]:
tfidf_sent_vectors[2000:2500]

[array([ 0.92862893,  0.13452719, -0.96678142,  0.26434511,  0.42144523,
         0.65803119, -0.05899041, -0.65341493,  1.41632409,  0.16615277,
         0.30923303,  0.51105144, -0.22983342,  0.70012177,  0.21935879,
         0.15729701, -0.71025658,  0.31760515, -0.71898171,  0.03365316,
         0.45362471,  0.16905394, -0.45528391,  0.51104203,  0.19232223,
        -0.02424515,  0.4695014 ,  0.23295665, -0.90702327,  0.01288731,
         0.18549689,  0.66019909, -0.9651284 ,  0.03883461,  0.08238942,
         0.73085807, -0.39718915, -0.1124608 ,  0.70906041, -0.30779181,
        -0.27112006, -0.71241152, -0.66654202,  0.62958621,  0.10343165,
         0.46488729,  0.70256545, -1.41841478, -0.01555521,  0.19257788]),
 array([ -2.06845352e-02,   6.86624242e-01,  -7.83003240e-01,
          2.35396060e-01,  -1.00309193e+00,  -1.23385600e-01,
         -8.75708430e-02,  -3.40976448e-01,   5.47083728e-01,
          6.33542722e-01,  -7.96325619e-01,  -6.24729386e-01,
          8.47496411

In [87]:
type(tfidf_sent_vectors)

list

In [88]:
len(tfidf_sent_vectors)

60000

In [28]:
with open('tfidf_sent_vect.pkl', 'wb') as pickle_file:
    pickle.dump(tfidf_sent_vectors, pickle_file)

### CREATING A FINAL DATAFRAME FOR ANALYSIS

In [29]:
final_analysis_file=pd.DataFrame(tfidf_sent_vectors)

In [30]:
final_analysis_file.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.177093,0.242106,0.056785,-0.108356,0.049898,0.365580,-0.230403,-0.282035,0.537327,0.315228,...,-0.073799,-0.360611,-0.688442,0.541719,0.343971,0.529262,0.202866,-0.640193,-0.069127,-0.008858
1,0.335403,0.269931,-0.439159,-0.235786,-0.585301,-0.044164,-0.489693,0.145499,-0.221961,0.081096,...,0.134597,-0.436086,-0.416241,-0.139369,0.128020,0.291305,-0.194745,-0.228211,0.087916,-0.058784
2,-0.071428,0.418853,0.030652,-0.623154,-0.008004,0.430800,-0.101884,0.443732,-0.397712,1.164939,...,-0.282017,-0.018403,-0.348121,-0.404755,1.214722,-0.179276,-1.291663,-0.897769,-0.466101,-0.292461
3,0.472152,0.529163,-0.007123,0.145510,-0.326849,-0.414855,-0.076689,-0.674746,-0.339968,0.669331,...,0.002660,-0.948620,-0.694976,0.906715,-0.106348,-0.490650,0.080084,-0.358495,-0.278575,0.099384
4,0.697582,0.652345,-0.234955,0.049723,0.136823,0.010255,-0.666608,-0.130333,0.921492,0.146175,...,-0.420204,-1.205891,-0.380073,0.235769,0.388563,0.243572,0.529020,-0.504115,-0.529403,0.222584


In [57]:
# del final_analysis_file["s_Score"]

### adding the column time and score

In [86]:
final_analysis_file["Time"]=final["Time"]

In [87]:
final_analysis_file["s_Score"]=final["Score"]

In [89]:
final_analysis_file.head(10)

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,Time,s_Score
0,0.177093,0.242106,0.056785,-0.108356,0.049898,0.365580,-0.230403,-0.282035,0.537327,0.315228,...,-0.688442,0.541719,0.343971,0.529262,0.202866,-0.640193,-0.069127,-0.008858,1300320000,1
1,0.335403,0.269931,-0.439159,-0.235786,-0.585301,-0.044164,-0.489693,0.145499,-0.221961,0.081096,...,-0.416241,-0.139369,0.128020,0.291305,-0.194745,-0.228211,0.087916,-0.058784,1345161600,1
2,-0.071428,0.418853,0.030652,-0.623154,-0.008004,0.430800,-0.101884,0.443732,-0.397712,1.164939,...,-0.348121,-0.404755,1.214722,-0.179276,-1.291663,-0.897769,-0.466101,-0.292461,1278374400,0
3,0.472152,0.529163,-0.007123,0.145510,-0.326849,-0.414855,-0.076689,-0.674746,-0.339968,0.669331,...,-0.694976,0.906715,-0.106348,-0.490650,0.080084,-0.358495,-0.278575,0.099384,1256169600,1
4,0.697582,0.652345,-0.234955,0.049723,0.136823,0.010255,-0.666608,-0.130333,0.921492,0.146175,...,-0.380073,0.235769,0.388563,0.243572,0.529020,-0.504115,-0.529403,0.222584,1348099200,1
5,0.624839,0.409909,-0.538451,-0.178786,-0.070633,0.076147,-0.498401,-0.003963,0.292649,-0.401479,...,-0.363425,0.124009,-0.164380,0.645197,-0.062170,-0.484241,-0.203763,-0.369825,1332460800,1
6,0.638531,0.264271,0.870590,-0.076104,0.352504,-0.963290,-0.761697,0.015832,0.330175,-0.637212,...,-0.845510,0.241261,-0.141848,-0.122690,0.155659,-1.155837,0.201811,-0.317373,1328227200,1
7,-0.042650,0.143509,0.179971,0.138971,-0.729724,0.280223,-0.756766,0.090949,0.017963,0.262992,...,-1.067473,-0.464743,0.738129,-0.073889,-0.554075,-0.107014,0.394616,-0.854593,1252713600,1
8,-0.696927,0.126945,0.162304,-0.894500,-0.192633,0.541183,0.104592,0.845583,-0.758714,1.031437,...,-0.023128,-0.852420,1.238973,-0.152407,-2.122015,-1.148228,-0.364399,-0.702815,1346284800,1
9,0.102286,0.368918,-0.268112,-0.265141,-0.190516,-0.084396,-0.573940,-0.117021,0.336471,0.349394,...,-0.765252,-0.339531,0.693140,0.172832,-0.486593,-0.313296,0.270683,-0.052448,1290556800,0


In [90]:
with open('final_analysis_file.pkl', 'wb') as pickle_file:
    pickle.dump(final_analysis_file, pickle_file)